# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from absa_models.ABSA_BERT_Dropout_BiLSTM_Linear import ABSA_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8
SEQ_LEN = 512

NO_RUNS = 10

In [10]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_bilstm_linear_512.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_bilstm_linear_512.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [14]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [15]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [16]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [17]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, bilstm_in_features=256, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []

    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses
    
    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ABSA/SemEval16 - Task 5 - Restaurants/plots/bert_pt_do_bilstm_lin/train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4025187492370605
Epoch: 0/2, Batch: 50/501, Loss: 0.10126699507236481
Epoch: 0/2, Batch: 100/501, Loss: 0.06132068112492561
Epoch: 0/2, Batch: 150/501, Loss: 0.029395051300525665
Epoch: 0/2, Batch: 200/501, Loss: 0.03187216818332672
Epoch: 0/2, Batch: 250/501, Loss: 0.024892928078770638
Epoch: 0/2, Batch: 300/501, Loss: 0.038860172033309937
Epoch: 0/2, Batch: 350/501, Loss: 0.04153978079557419
Epoch: 0/2, Batch: 400/501, Loss: 0.013941685669124126
Epoch: 0/2, Batch: 450/501, Loss: 0.0318756103515625
Epoch: 0/2, Batch: 500/501, Loss: 0.04233609139919281
Epoch: 1/2, Batch: 0/501, Loss: 0.02751404047012329
Epoch: 1/2, Batch: 50/501, Loss: 0.04281157627701759
Epoch: 1/2, Batch: 100/501, Loss: 0.055029042065143585
Epoch: 1/2, Batch: 150/501, Loss: 0.01889128051698208
Epoch: 1/2, Batch: 200/501, Loss: 0.02391939051449299
Epoch: 1/2, Batch: 250/501, Loss: 0.008717386983335018
Epoch: 1/2, Batch: 300/501, Loss: 0.02943866327404976
Epoch: 1/2, Batch: 35

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4605388641357422
Epoch: 0/2, Batch: 50/501, Loss: 0.08135326206684113
Epoch: 0/2, Batch: 100/501, Loss: 0.04323241859674454
Epoch: 0/2, Batch: 150/501, Loss: 0.09497629106044769
Epoch: 0/2, Batch: 200/501, Loss: 0.02573077753186226
Epoch: 0/2, Batch: 250/501, Loss: 0.043287668377161026
Epoch: 0/2, Batch: 300/501, Loss: 0.021370459347963333
Epoch: 0/2, Batch: 350/501, Loss: 0.040406323969364166
Epoch: 0/2, Batch: 400/501, Loss: 0.04029173403978348
Epoch: 0/2, Batch: 450/501, Loss: 0.03747021406888962
Epoch: 0/2, Batch: 500/501, Loss: 0.02020321413874626
Epoch: 1/2, Batch: 0/501, Loss: 0.029203880578279495
Epoch: 1/2, Batch: 50/501, Loss: 0.01524862740188837
Epoch: 1/2, Batch: 100/501, Loss: 0.020290762186050415
Epoch: 1/2, Batch: 150/501, Loss: 0.028817541897296906
Epoch: 1/2, Batch: 200/501, Loss: 0.01861860230565071
Epoch: 1/2, Batch: 250/501, Loss: 0.01780988648533821
Epoch: 1/2, Batch: 300/501, Loss: 0.03305967152118683
Epoch: 1/2, Batch: 3

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4292340278625488
Epoch: 0/2, Batch: 50/501, Loss: 0.09900355339050293
Epoch: 0/2, Batch: 100/501, Loss: 0.07043051719665527
Epoch: 0/2, Batch: 150/501, Loss: 0.03402223810553551
Epoch: 0/2, Batch: 200/501, Loss: 0.0355326309800148
Epoch: 0/2, Batch: 250/501, Loss: 0.06955531984567642
Epoch: 0/2, Batch: 300/501, Loss: 0.023589838296175003
Epoch: 0/2, Batch: 350/501, Loss: 0.026523659005761147
Epoch: 0/2, Batch: 400/501, Loss: 0.049054984003305435
Epoch: 0/2, Batch: 450/501, Loss: 0.02147614397108555
Epoch: 0/2, Batch: 500/501, Loss: 0.039840877056121826
Epoch: 1/2, Batch: 0/501, Loss: 0.03097626008093357
Epoch: 1/2, Batch: 50/501, Loss: 0.03717758134007454
Epoch: 1/2, Batch: 100/501, Loss: 0.0527292862534523
Epoch: 1/2, Batch: 150/501, Loss: 0.0469573475420475
Epoch: 1/2, Batch: 200/501, Loss: 0.01962929032742977
Epoch: 1/2, Batch: 250/501, Loss: 0.012425223365426064
Epoch: 1/2, Batch: 300/501, Loss: 0.02046097442507744
Epoch: 1/2, Batch: 350/5

...

Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4554989337921143
Epoch: 0/2, Batch: 50/501, Loss: 0.12241309136152267
Epoch: 0/2, Batch: 100/501, Loss: 0.049696680158376694
Epoch: 0/2, Batch: 150/501, Loss: 0.03871793672442436
Epoch: 0/2, Batch: 200/501, Loss: 0.04210398346185684
Epoch: 0/2, Batch: 250/501, Loss: 0.017495712265372276
Epoch: 0/2, Batch: 300/501, Loss: 0.04496234282851219
Epoch: 0/2, Batch: 350/501, Loss: 0.04759301245212555
Epoch: 0/2, Batch: 400/501, Loss: 0.034088630229234695
Epoch: 0/2, Batch: 450/501, Loss: 0.04974916949868202
Epoch: 0/2, Batch: 500/501, Loss: 0.04187829792499542
Epoch: 1/2, Batch: 0/501, Loss: 0.029374226927757263
Epoch: 1/2, Batch: 50/501, Loss: 0.030309826135635376
Epoch: 1/2, Batch: 100/501, Loss: 0.03221311420202255
Epoch: 1/2, Batch: 150/501, Loss: 0.016166601330041885
Epoch: 1/2, Batch: 200/501, Loss: 0.027664221823215485
Epoch: 1/2, Batch: 250/501, Loss: 0.011330223642289639
Epoch: 1/2, Batch: 300/501, Loss: 0.010523764416575432
Epoch: 1/2, Batch

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3432512283325195
Epoch: 0/2, Batch: 50/501, Loss: 0.08765968680381775
Epoch: 0/2, Batch: 100/501, Loss: 0.05349735543131828
Epoch: 0/2, Batch: 150/501, Loss: 0.051189932972192764
Epoch: 0/2, Batch: 200/501, Loss: 0.037613239139318466
Epoch: 0/2, Batch: 250/501, Loss: 0.0331394337117672
Epoch: 0/2, Batch: 300/501, Loss: 0.025327198207378387
Epoch: 0/2, Batch: 350/501, Loss: 0.022820213809609413
Epoch: 0/2, Batch: 400/501, Loss: 0.01685369946062565
Epoch: 0/2, Batch: 450/501, Loss: 0.017974765971302986
Epoch: 0/2, Batch: 500/501, Loss: 0.022418437525629997
Epoch: 1/2, Batch: 0/501, Loss: 0.026703352108597755
Epoch: 1/2, Batch: 50/501, Loss: 0.024249127134680748
Epoch: 1/2, Batch: 100/501, Loss: 0.01836538501083851
Epoch: 1/2, Batch: 150/501, Loss: 0.016584936529397964
Epoch: 1/2, Batch: 200/501, Loss: 0.04079845920205116
Epoch: 1/2, Batch: 250/501, Loss: 0.02032162994146347
Epoch: 1/2, Batch: 300/501, Loss: 0.008195714093744755
Epoch: 1/2, Batc

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [18]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.996141,0.996141,0.403832,0.996141,0.374396,0.996141,0.387514,375.954340
1,0.995516,0.995516,0.453195,0.995516,0.286984,0.995516,0.312106,371.122459
2,0.995480,0.995480,0.456639,0.995480,0.279457,0.995480,0.301057,370.249193
3,0.995156,0.995156,0.452042,0.995156,0.280801,0.995156,0.302923,369.659994
4,0.996609,0.996609,0.428249,0.996609,0.403716,0.996609,0.415043,369.725528
5,0.995363,0.995363,0.443893,0.995363,0.298327,0.995363,0.326931,370.989926
6,0.995930,0.995930,0.387220,0.995930,0.419665,0.995930,0.401744,370.260632
7,0.995875,0.995875,0.411216,0.995875,0.340618,0.995875,0.365849,370.920694
8,0.996168,0.996168,0.419668,0.996168,0.368269,0.996168,0.389290,357.120473
9,0.995859,0.995859,0.411273,0.995859,0.350967,0.995859,0.374053,348.858617


In [19]:
results.to_csv(STATS_OUTPUT)